# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [96]:
name = 'yannis'

## Imports

In [97]:
import pandas as pd
import numpy as np
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.helpers import *
from optimization.generation import *
from optimization.columnBuckReverse import *

In [98]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

## Constants

In [99]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}. Please verify.")

stringer_pitch = 200
effective_width = stringer_pitch/2

panel_element_length = 750/3
stringer_element_length = 750/3


Your personal data is: sigma_yield = 490, EModulus = 65241.07. Please verify.


# Importing necessary files 

## Import everything for panels 

In [100]:
# Import panel properties and match the elements to the respective panel
paneldf = pd.read_csv(f'../data/{name}/panel.csv')
panelPropertiesdf = pd.read_csv(f'../data/{name}/properties/panel_properties.csv', index_col=0)
panelPropertiesdf = panelPropertiesdf.drop(['mass', 'Component Name'], axis=1)
paneldf = pd.merge(paneldf, panelPropertiesdf, on='Element ID', how='left', suffixes=('_caller', '_other'))
paneldf = paneldf.drop(['sigmaXY', 'sigmaYY'], axis=1)
paneldf.head(5)

,Element ID,Component Name,sigmaXX,Load Case,thickness
0,1,panel1,-56.536,Subcase 1 (LC1),5.00
1,2,panel1,-70.203,Subcase 1 (LC1),5.00
2,3,panel1,-57.404,Subcase 1 (LC1),5.00
3,4,panel2,-80.353,Subcase 1 (LC1),4.94
4,5,panel2,-73.752,Subcase 1 (LC1),4.94


## Add a volume column to the panels 

In [101]:
paneldf['Volume'] = paneldf.apply(panel_element_volume, elementLength=panel_element_length, elementWidth=stringer_pitch, axis=1)
paneldf

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume
0,1,panel1,-56.536,Subcase 1 (LC1),5.00,125000.0
1,2,panel1,-70.203,Subcase 1 (LC1),5.00,125000.0
2,3,panel1,-57.404,Subcase 1 (LC1),5.00,125000.0
3,4,panel2,-80.353,Subcase 1 (LC1),4.94,123500.0
4,5,panel2,-73.752,Subcase 1 (LC1),4.94,123500.0
...,...,...,...,...,...,...
85,26,panel9,-24.502,Subcase 3 (LC3),4.94,123500.0
86,27,panel9,-28.539,Subcase 3 (LC3),4.94,123500.0
87,28,panel10,-20.656,Subcase 3 (LC3),5.00,125000.0
88,29,panel10,-23.322,Subcase 3 (LC3),5.00,125000.0


## Extract thicknesses left and right 

In [102]:
leftThickness = []
rightThickness = []
for i in range(0,9):
    leftThickness.append(paneldf['thickness'][0+3*i])
    rightThickness.append(paneldf['thickness'][3+3*i])
print(leftThickness)
print(rightThickness)

[np.float64(5.0), np.float64(4.94), np.float64(5.01), np.float64(5.29), np.float64(6.02), np.float64(6.02), np.float64(5.29), np.float64(5.01), np.float64(4.94)]
[np.float64(4.94), np.float64(5.01), np.float64(5.29), np.float64(6.02), np.float64(6.02), np.float64(5.29), np.float64(5.01), np.float64(4.94), np.float64(5.0)]


## Import everything for stringers 

In [103]:
# Open and match stringer properties 
stringerdf = pd.read_csv(f'../data/{name}/stringer.csv')
# Now add the stringer properties '../data/{name}/stringer_properties.csv
stringerPropertiesdf = pd.read_csv(f'../data/{name}/properties/stringer_properties.csv', index_col=0)
stringerPropertiesdf = stringerPropertiesdf.reset_index()
stringerPropertiesdf.rename(columns={'beamsects': 'Component Name', 'beamsect_dim1': 'dim1', 'beamsect_dim2': 'dim2', 'beamsect_dim3': 'dim3', 'beamsect_dim4': 'dim4'}, inplace=True)
# Add "stringer" prefix to Component Name
stringerPropertiesdf['Component Name'] = 'stringer' + stringerPropertiesdf['Component Name'].astype(str)
# Merge the dataframes
stringerdf = pd.merge(stringerdf, stringerPropertiesdf, on='Component Name', how='left', suffixes=('_caller', '_other'))
stringerdf

,Element ID,Component Name,sigmaXX,Load Case,dim1,dim2,dim3,dim4
0,40,stringer1,-57.426,Subcase 1 (LC1),36.90,1.69,27.82,15.41
1,41,stringer1,-52.850,Subcase 1 (LC1),36.90,1.69,27.82,15.41
2,42,stringer1,-59.080,Subcase 1 (LC1),36.90,1.69,27.82,15.41
3,43,stringer2,-61.209,Subcase 1 (LC1),38.59,1.51,28.82,14.51
4,44,stringer2,-64.293,Subcase 1 (LC1),38.59,1.51,28.82,14.51
...,...,...,...,...,...,...,...,...
76,62,stringer8,-21.359,Subcase 3 (LC3),38.59,1.51,28.82,14.51
77,63,stringer8,-15.171,Subcase 3 (LC3),38.59,1.51,28.82,14.51
78,64,stringer9,-22.649,Subcase 3 (LC3),36.90,1.69,27.82,15.41
79,65,stringer9,-17.558,Subcase 3 (LC3),36.90,1.69,27.82,15.41


## Add volume to the stringer elements

In [104]:
stringerdf['Volume']= stringerdf.apply(stringer_element_volume, elementLength = stringer_element_length, axis=1)
stringerdf.head(3)

,Element ID,Component Name,sigmaXX,Load Case,dim1,dim2,dim3,dim4,Volume
0,40,stringer1,-57.426,Subcase 1 (LC1),36.9,1.69,27.82,15.41,54527.85
1,41,stringer1,-52.850,Subcase 1 (LC1),36.9,1.69,27.82,15.41,54527.85
2,42,stringer1,-59.080,Subcase 1 (LC1),36.9,1.69,27.82,15.41,54527.85


# Now we rearrange the structure a bit

## First we split the 3 loadcases 

In [105]:
loadCase1dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 1 (LC1)']
loadCase2dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 2 (LC2)']
loadCase3dfPanel = paneldf[paneldf["Load Case"] == 'Subcase 3 (LC3)']
loadCase1dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 1 (LC1)']
loadCase2dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 2 (LC2)']
loadCase3dfStringer = stringerdf[stringerdf["Load Case"] == 'Subcase 3 (LC3)']
print(loadCase1dfPanel.head(5))
print(loadCase1dfStringer.head(5))

   Element ID Component Name  sigmaXX        Load Case  thickness    Volume
0           1         panel1  -56.536  Subcase 1 (LC1)       5.00  125000.0
1           2         panel1  -70.203  Subcase 1 (LC1)       5.00  125000.0
2           3         panel1  -57.404  Subcase 1 (LC1)       5.00  125000.0
3           4         panel2  -80.353  Subcase 1 (LC1)       4.94  123500.0
4           5         panel2  -73.752  Subcase 1 (LC1)       4.94  123500.0
   Element ID Component Name  sigmaXX        Load Case   dim1  dim2   dim3  \
0          40      stringer1  -57.426  Subcase 1 (LC1)  36.90  1.69  27.82   
1          41      stringer1  -52.850  Subcase 1 (LC1)  36.90  1.69  27.82   
2          42      stringer1  -59.080  Subcase 1 (LC1)  36.90  1.69  27.82   
3          43      stringer2  -61.209  Subcase 1 (LC1)  38.59  1.51  28.82   
4          44      stringer2  -64.293  Subcase 1 (LC1)  38.59  1.51  28.82   

    dim4    Volume  
0  15.41  54527.85  
1  15.41  54527.85  
2  15.41  54

# Now we need to combine the panels and the stringers

In [106]:
# For every loadcase
# Efficiently combine panels for load case 1 
panel_groups_1 = []
for i in range(1, 10):
    df1 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase1dfStringer[loadCase1dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_1.extend([df1, df2, df3])
lc1combined = pd.concat(panel_groups_1, ignore_index=True)
lc1combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-56.536,Subcase 1 (LC1),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-70.203,Subcase 1 (LC1),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-57.404,Subcase 1 (LC1),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-80.353,Subcase 1 (LC1),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-73.752,Subcase 1 (LC1),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-79.344,Subcase 1 (LC1),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-57.426,Subcase 1 (LC1),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
7,41,stringer1,-52.850,Subcase 1 (LC1),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
8,42,stringer1,-59.080,Subcase 1 (LC1),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
9,4,panel2,-80.353,Subcase 1 (LC1),4.94,123500.00,stiffener2,NaN,NaN,NaN,NaN


In [107]:
# Efficiently combine panels for load case 2
panel_groups_2 = []
for i in range(1, 10):
    df1 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase2dfStringer[loadCase2dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_2.extend([df1, df2, df3])
lc2combined = pd.concat(panel_groups_2, ignore_index=True)
lc2combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-58.010,Subcase 2 (LC2),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-70.451,Subcase 2 (LC2),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-58.878,Subcase 2 (LC2),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-80.031,Subcase 2 (LC2),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-73.084,Subcase 2 (LC2),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-79.022,Subcase 2 (LC2),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-63.302,Subcase 2 (LC2),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
7,41,stringer1,-67.686,Subcase 2 (LC2),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
8,42,stringer1,-64.956,Subcase 2 (LC2),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
9,4,panel2,-80.031,Subcase 2 (LC2),4.94,123500.00,stiffener2,NaN,NaN,NaN,NaN


In [108]:
# Efficiently combine panels for load case 3 
panel_groups_3 = []
for i in range(1, 10):
    df1 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i}'].copy()
    df2 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df3 = loadCase3dfStringer[loadCase3dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_3.extend([df1, df2, df3])
lc3combined = pd.concat(panel_groups_3, ignore_index=True)
lc3combined.head(10)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-17.197,Subcase 3 (LC3),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
1,2,panel1,-23.322,Subcase 3 (LC3),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
2,3,panel1,-20.656,Subcase 3 (LC3),5.00,125000.00,stiffener1,NaN,NaN,NaN,NaN
3,4,panel2,-28.539,Subcase 3 (LC3),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
4,5,panel2,-24.502,Subcase 3 (LC3),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
5,6,panel2,-24.515,Subcase 3 (LC3),4.94,123500.00,stiffener1,NaN,NaN,NaN,NaN
6,40,stringer1,-16.056,Subcase 3 (LC3),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
7,41,stringer1,-17.558,Subcase 3 (LC3),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
8,42,stringer1,-22.649,Subcase 3 (LC3),NaN,54527.85,stiffener1,36.9,1.69,27.82,15.41
9,4,panel2,-28.539,Subcase 3 (LC3),4.94,123500.00,stiffener2,NaN,NaN,NaN,NaN


In [109]:
# We fill the empty spaces with zero
lc1combined = lc1combined.fillna(0)
lc2combined = lc2combined.fillna(0)
lc3combined = lc3combined.fillna(0)
lc1combined

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4
0,1,panel1,-56.536,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00
1,2,panel1,-70.203,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00
2,3,panel1,-57.404,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00
3,4,panel2,-80.353,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00
4,5,panel2,-73.752,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
76,29,panel10,-70.203,Subcase 1 (LC1),5.00,125000.00,stiffener9,0.0,0.00,0.00,0.00
77,30,panel10,-56.536,Subcase 1 (LC1),5.00,125000.00,stiffener9,0.0,0.00,0.00,0.00
78,64,stringer9,-59.080,Subcase 1 (LC1),0.00,54527.85,stiffener9,36.9,1.69,27.82,15.41
79,65,stringer9,-52.850,Subcase 1 (LC1),0.00,54527.85,stiffener9,36.9,1.69,27.82,15.41


# Now we can aggregate the loadcases according to stiffeners

## Multiply volume and stress together for averaging 

In [110]:
lc1combined['XX * Volume'] = lc1combined['sigmaXX'] * lc1combined['Volume']
lc2combined['XX * Volume'] = lc2combined['sigmaXX'] * lc2combined['Volume']
lc3combined['XX * Volume'] = lc3combined['sigmaXX'] * lc3combined['Volume']
lc1combined.head(9)

,Element ID,Component Name,sigmaXX,Load Case,thickness,Volume,Stiffener,dim1,dim2,dim3,dim4,XX * Volume
0,1,panel1,-56.536,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00,-7.067000e+06
1,2,panel1,-70.203,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00,-8.775375e+06
2,3,panel1,-57.404,Subcase 1 (LC1),5.00,125000.00,stiffener1,0.0,0.00,0.00,0.00,-7.175500e+06
3,4,panel2,-80.353,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00,-9.923596e+06
4,5,panel2,-73.752,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00,-9.108372e+06
5,6,panel2,-79.344,Subcase 1 (LC1),4.94,123500.00,stiffener1,0.0,0.00,0.00,0.00,-9.798984e+06
6,40,stringer1,-57.426,Subcase 1 (LC1),0.00,54527.85,stiffener1,36.9,1.69,27.82,15.41,-3.131316e+06
7,41,stringer1,-52.850,Subcase 1 (LC1),0.00,54527.85,stiffener1,36.9,1.69,27.82,15.41,-2.881797e+06
8,42,stringer1,-59.080,Subcase 1 (LC1),0.00,54527.85,stiffener1,36.9,1.69,27.82,15.41,-3.221505e+06


## Load case 1 

In [111]:
lc1combined = lc1combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc1combined['sigma_XX_avg'] = lc1combined['XX * Volume'] / lc1combined['Volume']
lc1combined['tLeft'] = leftThickness
lc1combined['tRight'] = rightThickness
lc1combined = lc1combined.drop(['XX * Volume'], axis=1)
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-67.192333,5.00,4.94
stiffener2,895740.000,38.59,1.51,28.82,14.51,-74.711561,4.94,5.01
stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.710928,5.01,5.29
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-67.245222,5.29,6.02
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-65.412732,6.02,6.02
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-67.245222,6.02,5.29
stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.710928,5.29,5.01
stiffener8,895740.000,38.59,1.51,28.82,14.51,-74.711561,5.01,4.94
stiffener9,909083.550,36.90,1.69,27.82,15.41,-67.192333,4.94,5.00


## Load case 2

In [112]:
lc2combined = lc2combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc2combined['sigma_XX_avg'] = lc2combined['XX * Volume'] / lc2combined['Volume']
lc2combined = lc2combined.drop(['XX * Volume'], axis=1)
lc2combined['tLeft'] = leftThickness
lc2combined['tRight'] = rightThickness
lc2combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-69.048328,5.00,4.94
stiffener2,895740.000,38.59,1.51,28.82,14.51,-74.129605,4.94,5.01
stiffener3,929182.575,37.42,1.59,29.05,15.34,-71.616114,5.01,5.29
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-67.192978,5.29,6.02
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-65.339701,6.02,6.02
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-67.192978,6.02,5.29
stiffener7,929182.575,37.42,1.59,29.05,15.34,-71.616114,5.29,5.01
stiffener8,895740.000,38.59,1.51,28.82,14.51,-74.129605,5.01,4.94
stiffener9,909083.550,36.90,1.69,27.82,15.41,-69.048328,4.94,5.00


## Load case 3

In [113]:
lc3combined = lc3combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc3combined['sigma_XX_avg'] = lc3combined['XX * Volume'] / lc3combined['Volume']
lc3combined = lc3combined.drop(['XX * Volume'], axis=1)
lc3combined['tLeft'] = leftThickness
lc3combined['tRight'] = rightThickness
lc3combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,909083.550,36.90,1.69,27.82,15.41,-22.322416,5.00,4.94
stiffener2,895740.000,38.59,1.51,28.82,14.51,-24.820464,4.94,5.01
stiffener3,929182.575,37.42,1.59,29.05,15.34,-23.823444,5.01,5.29
stiffener4,1007645.850,37.84,1.62,28.83,14.96,-22.339858,5.29,6.02
stiffener5,1055089.650,37.80,1.53,29.90,15.05,-21.731012,6.02,6.02
stiffener6,1007645.850,37.84,1.62,28.83,14.96,-22.339858,6.02,5.29
stiffener7,929182.575,37.42,1.59,29.05,15.34,-23.823444,5.29,5.01
stiffener8,895740.000,38.59,1.51,28.82,14.51,-24.820464,5.01,4.94
stiffener9,909083.550,36.90,1.69,27.82,15.41,-22.322416,4.94,5.00


# Now we add Cross-Section Properties of the combined skin and hat stringer crosssection 

In [114]:
# Load case 1
lc1combined[['I_yy', 'areaTot', 'VolumeTot']] = lc1combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 2
lc2combined[['I_yy', 'areaTot', 'VolumeTot']] = lc2combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
# Load case 3
lc3combined[['I_yy', 'areaTot', 'VolumeTot']] = lc3combined.apply(crosssectional_properties_hat_skin_row, stringer_pitch=stringer_pitch, stringer_depth = stringer_element_length*3,
                                                                    axis=1, result_type='expand')
lc1combined

KeyError: 'thickness'

# Now we calculate the columnbuckling with Euler Johnson 

In [ ]:
# Load case 1EModulus, DIM1, DIM2, DIM3, sigma_yield, r
lc1combined['sigma_crip'] = lc1combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
# Load case 2
lc2combined['sigma_crip'] = lc2combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
#Load case 3
lc3combined['sigma_crip'] = lc3combined.apply(lambda row: sigma_crip(EModulus= EModulus, DIM1=row['dim1'], DIM2=row['dim2'], DIM3=row['dim3'], sigma_yield = sigma_yield, r=0 ),
                                                axis=1)
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip
Stiffener,,,,,,,,,,,
stiffener1,909083.550,5.00,36.90,1.69,27.82,15.41,-67.192333,117231.831115,1218.1114,913583.550,408.093629
stiffener2,895740.000,5.01,38.59,1.51,28.82,14.51,-74.711561,120751.137376,1201.3200,900990.000,359.901302
stiffener3,929182.575,5.29,37.42,1.59,29.05,15.34,-71.710928,119888.392953,1266.9101,950182.575,384.331012
stiffener4,1007645.850,6.02,37.84,1.62,28.83,14.96,-67.245222,129905.900650,1416.5278,1062395.850,387.614361
stiffener5,1055089.650,6.02,37.80,1.53,29.90,15.05,-65.412732,125993.523988,1406.7862,1055089.650,367.757298
stiffener6,1007645.850,6.02,37.84,1.62,28.83,14.96,-67.245222,129905.900650,1416.5278,1062395.850,387.614361
stiffener7,929182.575,5.29,37.42,1.59,29.05,15.34,-71.710928,119888.392953,1266.9101,950182.575,384.331012
stiffener8,895740.000,5.01,38.59,1.51,28.82,14.51,-74.711561,120751.137376,1201.3200,900990.000,359.901302
stiffener9,909083.550,5.00,36.90,1.69,27.82,15.41,-67.192333,117231.831115,1218.1114,913583.550,408.093629


## Add lambda_crit to the loadcases

In [ ]:
lc1combined['lambda_crit'] = lc1combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc2combined['lambda_crit'] = lc2combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)
lc3combined['lambda_crit'] = lc3combined.apply(lambda row: lambda_crit(EModulus, row['sigma_crip'], personal_data[0]), axis=1)

## Add lambda to the loadcases

In [ ]:
lc1combined['lambda'] = lc1combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc2combined['lambda'] = lc2combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc3combined['lambda'] = lc3combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)

## Add radius of gyration to the loadcases

In [ ]:
lc1combined['r_gyr'] = lc1combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc2combined['r_gyr'] = lc2combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc3combined['r_gyr'] = lc3combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)

## Now calculate the critical stress & Reserve Factor

In [ ]:
lc1combined[['sigma_crit', 'Reserve Factor']] = lc1combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc2combined[['sigma_crit', 'Reserve Factor']] = lc2combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc3combined[['sigma_crit', 'Reserve Factor']] = lc3combined.apply(chooseBuckling, EModulus=EModulus, sigma_yield=sigma_yield, axis=1, result_type='expand')
lc1combined

,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
Stiffener,,,,,,,,,,,,,,,,
stiffener1,909083.550,5.00,36.90,1.69,27.82,15.41,-67.192333,117231.831115,1218.1114,913583.550,408.093629,56.175313,76.450793,9.810232,110.168347,1.093065
stiffener2,895740.000,5.01,38.59,1.51,28.82,14.51,-74.711561,120751.137376,1201.3200,900990.000,359.901302,59.818249,74.807476,10.025736,115.061708,1.026719
stiffener3,929182.575,5.29,37.42,1.59,29.05,15.34,-71.710928,119888.392953,1266.9101,950182.575,384.331012,57.885887,77.098440,9.727823,108.325234,1.007055
stiffener4,1007645.850,6.02,37.84,1.62,28.83,14.96,-67.245222,129905.900650,1416.5278,1062395.850,387.614361,57.640200,78.317601,9.576391,104.978913,1.040757
stiffener5,1055089.650,6.02,37.80,1.53,29.90,15.05,-65.412732,125993.523988,1406.7862,1055089.650,367.757298,59.175884,79.250352,9.463680,102.522318,1.044876
stiffener6,1007645.850,6.02,37.84,1.62,28.83,14.96,-67.245222,129905.900650,1416.5278,1062395.850,387.614361,57.640200,78.317601,9.576391,104.978913,1.040757
stiffener7,929182.575,5.29,37.42,1.59,29.05,15.34,-71.710928,119888.392953,1266.9101,950182.575,384.331012,57.885887,77.098440,9.727823,108.325234,1.007055
stiffener8,895740.000,5.01,38.59,1.51,28.82,14.51,-74.711561,120751.137376,1201.3200,900990.000,359.901302,59.818249,74.807476,10.025736,115.061708,1.026719
stiffener9,909083.550,5.00,36.90,1.69,27.82,15.41,-67.192333,117231.831115,1218.1114,913583.550,408.093629,56.175313,76.450793,9.810232,110.168347,1.093065


# Generate Score output 

In [ ]:

lc1combined = lc1combined.reset_index()
lc2combined = lc2combined.reset_index()
lc3combined = lc3combined.reset_index()

evaluateDf = pd.concat([lc1combined, lc2combined, lc3combined])
evaluateDf = evaluateDf.reset_index()
evaluateDf = evaluateDf.drop(['index'], axis=1)
updateDF = evaluateDf.copy()
evaluateDf

,Stiffener,Volume,thickness,dim1,dim2,dim3,dim4,sigma_XX_avg,I_yy,areaTot,VolumeTot,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
0,stiffener1,909083.550,5.00,36.90,1.69,27.82,15.41,-67.192333,117231.831115,1218.1114,913583.550,408.093629,56.175313,76.450793,9.810232,110.168347,1.093065
1,stiffener2,895740.000,5.01,38.59,1.51,28.82,14.51,-74.711561,120751.137376,1201.3200,900990.000,359.901302,59.818249,74.807476,10.025736,115.061708,1.026719
2,stiffener3,929182.575,5.29,37.42,1.59,29.05,15.34,-71.710928,119888.392953,1266.9101,950182.575,384.331012,57.885887,77.098440,9.727823,108.325234,1.007055
3,stiffener4,1007645.850,6.02,37.84,1.62,28.83,14.96,-67.245222,129905.900650,1416.5278,1062395.850,387.614361,57.640200,78.317601,9.576391,104.978913,1.040757
4,stiffener5,1055089.650,6.02,37.80,1.53,29.90,15.05,-65.412732,125993.523988,1406.7862,1055089.650,367.757298,59.175884,79.250352,9.463680,102.522318,1.044876
5,stiffener6,1007645.850,6.02,37.84,1.62,28.83,14.96,-67.245222,129905.900650,1416.5278,1062395.850,387.614361,57.640200,78.317601,9.576391,104.978913,1.040757
6,stiffener7,929182.575,5.29,37.42,1.59,29.05,15.34,-71.710928,119888.392953,1266.9101,950182.575,384.331012,57.885887,77.098440,9.727823,108.325234,1.007055
7,stiffener8,895740.000,5.01,38.59,1.51,28.82,14.51,-74.711561,120751.137376,1201.3200,900990.000,359.901302,59.818249,74.807476,10.025736,115.061708,1.026719
8,stiffener9,909083.550,5.00,36.90,1.69,27.82,15.41,-67.192333,117231.831115,1218.1114,913583.550,408.093629,56.175313,76.450793,9.810232,110.168347,1.093065
9,stiffener1,909083.550,5.00,36.90,1.69,27.82,15.41,-69.048328,117231.831115,1218.1114,913583.550,408.093629,56.175313,76.450793,9.810232,110.168347,1.063683


In [ ]:
evaluateDf = evaluateDf.drop(['Volume' ,'thickness' , 'sigma_XX_avg', 'I_yy',  'areaTot', 'VolumeTot', 'sigma_crip', 'lambda_crit', 'lambda', 'r_gyr', 'sigma_crit'], axis =1)
evaluateDf['score'] = evaluateDf.apply(rf_score, axis=1)
evaluateDf

,Stiffener,dim1,dim2,dim3,dim4,Reserve Factor,score
0,stiffener1,36.90,1.69,27.82,15.41,1.093065,0.046364
1,stiffener2,38.59,1.51,28.82,14.51,1.026719,0.013550
2,stiffener3,37.42,1.59,29.05,15.34,1.007055,0.046108
3,stiffener4,37.84,1.62,28.83,14.96,1.040757,0.002136
4,stiffener5,37.80,1.53,29.90,15.05,1.044876,0.000656
5,stiffener6,37.84,1.62,28.83,14.96,1.040757,0.002136
6,stiffener7,37.42,1.59,29.05,15.34,1.007055,0.046108
7,stiffener8,38.59,1.51,28.82,14.51,1.026719,0.013550
8,stiffener9,36.90,1.69,27.82,15.41,1.093065,0.046364
9,stiffener1,36.90,1.69,27.82,15.41,1.063683,0.004681


## Extract score and parameters

In [ ]:
score = evaluateDf['score'].sum()
dimensions = extractDimensions(evaluateDf)

evaluateDf = pd.DataFrame({
    'stringer Parameters': [dimensions],
    'score': [score],
    'minRF':[evaluateDf['Reserve Factor'].min()]
})
evaluateDf

,stringer Parameters,score,minRF
0,"[[36.9, 1.69, 27.82, 15.41], [38.59, 1.51, 28....",982.369525,1.007055


## Output stringer score

In [ ]:
evaluateDf.to_csv(f'../data/{name}/output/stringerScore.csv', index=False)

# Reverse Engineering 

## Collect all dimension updates 

In [ ]:

updateDF[['new_dim1', 'new_dim3','new_dim4']] = updateDF.apply(reverseAllDims, EModulus=EModulus,
                                                                            stringerPitch=stringer_pitch,
                                                                            length=stringer_element_length*3,
                                                                             RFgoal=0.9, axis=1, result_type='expand')
updateDF

ValueError: Columns must be same length as key

## Cleanup dataframe

In [ ]:
updateDF = updateDF.drop(['Volume' ,'thickness' , 'sigma_XX_avg', 'I_yy',  'areaTot', 'VolumeTot', 'sigma_crip', 'lambda_crit', 'lambda', 'r_gyr', 'sigma_crit'], axis =1)

## Keep only the largest values 

In [ ]:
updateDF = updateDF.groupby('Stiffener').agg({
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
    'new_dim1': 'max',
    'new_dim3': 'max',
    'new_dim4': 'max'
})
updateDF


,dim1,dim2,dim3,dim4,new_dim1,new_dim3,new_dim4
Stiffener,,,,,,,
stiffener1,25.0,2.0,20.0,15.0,30.0579320589217,43.1505473439788,283.835250038476
stiffener2,25.0,2.0,20.0,15.0,31.3098134595724,50.6665199089262,227.682754060223
stiffener3,25.0,2.0,20.0,15.0,31.3263438132660,50.7717567394411,227.134163821461
stiffener4,25.0,2.0,20.0,15.0,31.2340415900446,50.1862357087708,230.207156112507
stiffener5,25.0,2.0,20.0,15.0,31.1845082876813,49.8741188206725,231.866527853879
stiffener6,25.0,2.0,20.0,15.0,31.2340415900446,50.1862357087708,230.207156112507
stiffener7,25.0,2.0,20.0,15.0,31.3263438132660,50.7717567394411,227.134163821461
stiffener8,25.0,2.0,20.0,15.0,31.3098134595724,50.6665199089262,227.682754060223
stiffener9,25.0,2.0,20.0,15.0,30.0579320589217,43.1505473439788,283.835250038476


## Add differnces 

In [ ]:
updateDF['diff_dim1'] = (updateDF['new_dim1'] - updateDF['dim1']).astype(float)
updateDF['diff_dim3'] = (updateDF['new_dim3'] - updateDF['dim3']). astype(float)
updateDF['diff_dim4'] = (updateDF['new_dim4'] - updateDF['dim4']).astype(float)

#Convert type 
updateDF['new_dim1'] = updateDF['new_dim1'].astype(float)
updateDF['new_dim3'] = updateDF['new_dim3'].astype(float)
updateDF['new_dim4'] = updateDF['new_dim4'].astype(float)


updateDF = updateDF.round(rounding_digits)
updateDF

,dim1,dim2,dim3,dim4,new_dim1,new_dim3,new_dim4,diff_dim1,diff_dim3,diff_dim4
Stiffener,,,,,,,,,,
stiffener1,25.0,2.0,20.0,15.0,30.06,43.15,283.84,5.06,23.15,268.84
stiffener2,25.0,2.0,20.0,15.0,31.31,50.67,227.68,6.31,30.67,212.68
stiffener3,25.0,2.0,20.0,15.0,31.33,50.77,227.13,6.33,30.77,212.13
stiffener4,25.0,2.0,20.0,15.0,31.23,50.19,230.21,6.23,30.19,215.21
stiffener5,25.0,2.0,20.0,15.0,31.18,49.87,231.87,6.18,29.87,216.87
stiffener6,25.0,2.0,20.0,15.0,31.23,50.19,230.21,6.23,30.19,215.21
stiffener7,25.0,2.0,20.0,15.0,31.33,50.77,227.13,6.33,30.77,212.13
stiffener8,25.0,2.0,20.0,15.0,31.31,50.67,227.68,6.31,30.67,212.68
stiffener9,25.0,2.0,20.0,15.0,30.06,43.15,283.84,5.06,23.15,268.84


## Output of Reverse 

In [ ]:
updateDF.to_csv(f'../data/{name}/output/newStringerDims.csv', index=False)

# Cleanup data for output 

## Drop unenessacry columns 

In [ ]:
lc1combined = lc1combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc2combined = lc2combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc3combined = lc3combined.drop(['Volume', 'thickness', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'VolumeTot'], axis=1)
lc1combined

,Stiffener,sigma_XX_avg,I_yy,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
0,stiffener1,-81.279844,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.410701
1,stiffener2,-90.999022,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.366836
2,stiffener3,-91.105554,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.366407
3,stiffener4,-90.511651,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.368811
4,stiffener5,-90.193892,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.370110
5,stiffener6,-90.511651,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.368811
6,stiffener7,-91.105554,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.366407
7,stiffener8,-90.999022,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.366836
8,stiffener9,-81.279844,43392.340054,490,51.265756,113.39935,6.613795,50.072548,0.410701


## ROUND & Add together the load cases 

In [ ]:
# Rename colums for concat
lc1combined = lc1combined.rename(columns={'sigma_XX_avg':'XX_avg_LC1', 'sigma_crit':'sigma_crit_LC1', 'Reserve Factor':'RF_LC1', 'sigma_crip':'sigma_crip_LC1'})
lc2combined = lc2combined.rename(columns={'sigma_XX_avg':'XX_avg_LC2', 'sigma_crit':'sigma_crit_LC2', 'Reserve Factor':'RF_LC2', 'sigma_crip':'sigma_crip_LC2'})
lc3combined = lc3combined.rename(columns={'sigma_XX_avg':'XX_avg_LC3', 'sigma_crit':'sigma_crit_LC3', 'Reserve Factor':'RF_LC3', 'sigma_crip':'sigma_crip_LC3'})

In [ ]:
outputdf = pd.concat([lc1combined,lc2combined,lc3combined], axis = 1)
outputdf = outputdf.round(rounding_digits)
# After concatenation, keep only the first column of cross section propertries and drop the rest
outputdf['I_y'] = outputdf.filter(like='I_yy').iloc[:, 0]  # Take the first I_yy column
outputdf['Lambda'] = outputdf.filter(like='lambda').iloc[:, 1]  # Take the first lambda column
outputdf['Lambda_crit'] = outputdf.filter(like='lambda_crit').iloc[:, 0]  # Take the first I_yy column
outputdf['R_gyr'] = outputdf.filter(like='r_gyr').iloc[:, 0]  # Take the first I_yy column
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('I_yy')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('lambda')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('r_gyr')])
outputdf.head(10)

,Stiffener,XX_avg_LC1,sigma_crip_LC1,sigma_crit_LC1,RF_LC1,Stiffener,XX_avg_LC2,sigma_crip_LC2,sigma_crit_LC2,RF_LC2,Stiffener,XX_avg_LC3,sigma_crip_LC3,sigma_crit_LC3,RF_LC3,I_y,Lambda,Lambda_crit,R_gyr
0,stiffener1,-81.28,490,50.07,0.41,stiffener1,-83.15,490,50.07,0.40,stiffener1,-27.00,490,50.07,1.24,43392.34,113.4,51.27,6.61
1,stiffener2,-91.00,490,50.07,0.37,stiffener2,-90.42,490,50.07,0.37,stiffener2,-30.23,490,50.07,1.10,43392.34,113.4,51.27,6.61
2,stiffener3,-91.11,490,50.07,0.37,stiffener3,-91.07,490,50.07,0.37,stiffener3,-30.27,490,50.07,1.10,43392.34,113.4,51.27,6.61
3,stiffener4,-90.51,490,50.07,0.37,stiffener4,-90.42,490,50.07,0.37,stiffener4,-30.07,490,50.07,1.11,43392.34,113.4,51.27,6.61
4,stiffener5,-90.19,490,50.07,0.37,stiffener5,-90.19,490,50.07,0.37,stiffener5,-29.96,490,50.07,1.11,43392.34,113.4,51.27,6.61
5,stiffener6,-90.51,490,50.07,0.37,stiffener6,-90.42,490,50.07,0.37,stiffener6,-30.07,490,50.07,1.11,43392.34,113.4,51.27,6.61
6,stiffener7,-91.11,490,50.07,0.37,stiffener7,-91.07,490,50.07,0.37,stiffener7,-30.27,490,50.07,1.10,43392.34,113.4,51.27,6.61
7,stiffener8,-91.00,490,50.07,0.37,stiffener8,-90.42,490,50.07,0.37,stiffener8,-30.23,490,50.07,1.10,43392.34,113.4,51.27,6.61
8,stiffener9,-81.28,490,50.07,0.41,stiffener9,-83.15,490,50.07,0.40,stiffener9,-27.00,490,50.07,1.24,43392.34,113.4,51.27,6.61


# Generate output file 

In [ ]:
outputdf.to_excel(f'../data/{name}/output/processed_f.xlsx')